# Importing Packages

In [2]:
# packages for data collection and cleaning
import requests
import json
from io import StringIO
from bs4 import BeautifulSoup as bs
from datetime import date, timedelta
import covidcast
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)

# packages for mapping
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl



# Data Collection and Exploratory Data Analysis


## NYT Mask Wearing Survey Data

- This data comes from a large number of interviews conducted online by the firm Dynata, which asked the question to obtain 250,000 survey responses between July 2 and July 14, enough data to provide estimates more detailed than the state level. 
- This survey was conducted for a single run and there are no plans for a follow-up study.
- Specifically, each participant was asked the following question: 

<center><strong>How often do you wear a mask in public when you expect to be within six feet of another person?</strong></center>

- Here are the definitions for the column headings:

    - **COUNTYFP**: The county FIPS code.
    - **NEVER**: The estimated share of people in this county who would say never in response to the question: “How often do you wear a mask in public when you expect to be within six feet of another person?”
    - **RARELY**: The estimated share of people in this county who would say rarely
    - **SOMETIMES**: The estimated share of people in this county who would say sometimes
    - **FREQUENTLY**: The estimated share of people in this county who would say frequently
    - **ALWAYS**: The estimated share of people in this county who would say always

 - In their analysis, they assumed the following to be true:  ALWAYS --> 100%, FREQUENTLY --> 80%, SOMETIMES --> 50%, RARELY --> 20%, NEVER --> 0%




In [3]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv'
s = requests.get(url).text
nymask = pd.read_csv(StringIO(s))
nymask.COUNTYFP = nymask.COUNTYFP.astype(str)
nymask.COUNTYFP = np.where(nymask['COUNTYFP'].str.len() == 4, '0' + nymask.COUNTYFP, nymask.COUNTYFP)
nymask.columns = ['FIPS', 'NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS']

,FIPS,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,01001,0.053,0.074,0.134,0.295,0.444
1,01003,0.083,0.059,0.098,0.323,0.436
2,01005,0.067,0.121,0.120,0.201,0.491
3,01007,0.020,0.034,0.096,0.278,0.572
4,01009,0.053,0.114,0.180,0.194,0.459


In [5]:
nymask.describe()

,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
count,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000
mean,0.079940,0.082919,0.121318,0.207725,0.508094
std,0.058538,0.055464,0.058011,0.063571,0.152191
min,0.000000,0.000000,0.001000,0.029000,0.115000
25%,0.034000,0.040000,0.079000,0.164000,0.393250
50%,0.068000,0.073000,0.115000,0.204000,0.497000
75%,0.113000,0.115000,0.156000,0.247000,0.613750
max,0.432000,0.384000,0.422000,0.549000,0.889000


<img src='images/fb_mask_data.png'>

## Carnegie Mellon Mask Wearing Facebook Survey Data

- In conjunction with Facebook and survey firm, the Carnegie Mellon study collected 1,220,000 valid responses from respondents across the US.

- The survey involves questions about symptoms, mask wearing, testing, and the other important topics in relation to COVID-19 as described above, along with demographic details about the respondent. 
    - These demographics include age, gender, race, occupation, and education, allowing us to understand how different groups have been affected and which groups are currently most vulnerable to COVID-19. 
- The two relevant questions in relation to face masks are the following:

    1. In the past 5 days, did you wear a mask most or all of the time in public?
    2. In the past 7 days, when you were in public places where social distancing is not possible, did most or all other people wear masks?

- The four signals available for data collection through Covidcast are:
    1. `**smoothed_wearing_mask**`
    2. `**smoothed_others_masked**`
    3. `**smoothed_wwearing_mask**`
    4. `**smoothed_wothers_masked**`

- The smoothed versions of all the `fb-survey` signals (with `smoothed_` prefix) are calculated using seven day pooling.
- The weighting versions (with the `smoothed_w` prefix) adjust the data to be representative of the US population, adjusting both for:

    1. the differences between the US population and US Facebook users (according to a state-by-age-gender stratification of the US population from the 2018 Census March Supplement)
    2. the propensity of a Facebook user to take our survey in the first place

In [20]:
# create variable to hold date object for two days ago
two_days_ago = date.today() - timedelta(days=2)
# get the most recent survey data
mask_ind = covidcast.signal("fb-survey", "smoothed_wearing_mask", date(2020, 10, 1), two_days_ago, "county")
mask_oth = covidcast.signal('fb-survey', 'smoothed_others_masked', date(2020, 11, 24),two_days_ago, "county")
maskw_ind = covidcast.signal('fb-survey', 'smoothed_wwearing_mask', date(2020, 10, 1), two_days_ago, "county")
maskw_oth = covidcast.signal('fb-survey', 'smoothed_wothers_masked', date(2020, 10, 1), two_days_ago, "county")

In [21]:
mask_ind.head()

,geo_value,signal,time_value,issue,lag,value,stderr,sample_size,geo_type,data_source
0,01000,smoothed_wearing_mask,2020-10-01,2020-12-09,69,79.048264,1.045944,1513.8950,county,fb-survey
1,01003,smoothed_wearing_mask,2020-10-01,2020-12-09,69,73.866307,2.968477,219.0684,county,fb-survey
2,01073,smoothed_wearing_mask,2020-10-01,2020-12-09,69,88.893937,1.822515,297.2281,county,fb-survey
3,01089,smoothed_wearing_mask,2020-10-01,2020-12-09,69,90.361381,1.421118,431.2580,county,fb-survey
4,01097,smoothed_wearing_mask,2020-10-01,2020-12-09,69,80.325870,2.119979,351.6316,county,fb-survey


In [22]:
mask_ind.describe()

,lag,value,stderr,sample_size
count,55440.000000,55440.000000,55440.000000,55440.000000
mean,24.921735,90.411525,1.803575,406.643938
std,21.998323,5.897936,0.860089,466.253229
min,0.000000,51.314471,0.251097,100.000000
25%,5.000000,87.423473,1.104658,148.239525
50%,16.000000,91.851871,1.672332,240.730900
75%,45.000000,94.818840,2.386508,464.951650
max,69.000000,99.677919,4.965824,4677.966900


In [23]:
mask_oth.head()

,geo_value,signal,time_value,issue,lag,value,stderr,sample_size,geo_type,data_source
0,01000,smoothed_others_masked,2020-11-24,2020-12-09,15,63.732745,2.362805,414.0201,county,fb-survey
1,04000,smoothed_others_masked,2020-11-24,2020-12-09,15,69.457396,3.936373,136.9090,county,fb-survey
2,04013,smoothed_others_masked,2020-11-24,2020-12-09,15,82.923867,2.051720,336.3820,county,fb-survey
3,04019,smoothed_others_masked,2020-11-24,2020-12-09,15,84.678313,2.964860,147.5946,county,fb-survey
4,05000,smoothed_others_masked,2020-11-24,2020-12-09,15,64.576451,2.575665,344.8160,county,fb-survey


In [24]:
mask_oth.describe()

,lag,value,stderr,sample_size
count,25061.000000,25061.000000,25061.000000,25061.000000
mean,5.478752,81.796308,2.287603,410.989417
std,2.169420,12.298306,1.046096,475.412758
min,0.000000,24.554959,0.418443,100.000000
25%,5.000000,75.538850,1.423795,149.000000
50%,5.000000,85.466091,2.106830,238.206900
75%,5.000000,91.157208,3.085610,475.804800
max,15.000000,98.750000,4.992825,4377.102200


In [26]:
# remove data that isn't at the county level
mask_ind = mask_ind.loc[~mask_ind.geo_value.str.endswith('000')]
mask_oth = mask_oth.loc[~mask_oth.geo_value.str.endswith('000')]
# check number of counties represented 
print(mask_ind.geo_value.value_counts().shape)
print(mask_oth.geo_value.value_counts().shape)

mask_ind.rename(columns = {'geo_value': 'FIPS'}, inplace=True)
mask_oth.rename(columns = {'geo_value': 'FIPS'}, inplace=True)
mask_ind.FIPS = mask_ind.FIPS.apply(lambda x: x.zfill(5))
mask_oth.FIPS = mask_oth.FIPS.apply(lambda x: x.zfill(5))

(680,)
(659,)


In [28]:
# pare down datasets to just columns of interest
mask_ind_means = pd.DataFrame(mask_ind.groupby(['FIPS'])['value'].mean()).reset_index()
mask_ind_means.columns = ['FIPS', 'ind_mask']
mask_oth_means = pd.DataFrame(mask_oth.groupby(['FIPS'])['value'].mean()).reset_index()
mask_oth_means.columns = ['FIPS', 'oth_mask']
# merge both datasets together
mask_means = mask_ind_means.merge(mask_oth_means, on = 'FIPS', how = 'outer')

In [29]:
mask_means.head()

,FIPS,ind_mask,oth_mask
0,01003,79.745931,52.865139
1,01051,88.255355,65.708307
2,01069,85.630290,57.096947
3,01073,91.482391,79.565024
4,01081,90.116112,70.572951


<img src='images/delphi_dec.png'>

Data from Delphi COVIDcast. Obtained via the Delphi Epidata API. https://cmu-delphi.github.io/delphi-epidata/api/covidcast.html

## Johns Hopkins COVID Data Geopandas

- This geopandas feature layer contains the most up-to-date COVID-19 cases for the US. 
- Data is pulled from the Coronavirus COVID-19 Global Cases by the CSSE at Johns Hopkins University, the Red Cross, the Census American Community Survey, and the Bureau of Labor and Statistics, and aggregated at the US county level. 


In [53]:
geo = gpd.read_file('https://opendata.arcgis.com/datasets/4cb598ae041348fb92270f102a6783cb_0.geojson')
print(geo.shape)
geo.head()

(3331, 88)


,OBJECTID,Countyname,ST_Name,ST_Abbr,ST_ID,FIPS,FatalityRa,Confirmedb,DeathsbyPo,PCTPOVALL_,Unemployme,Med_HH_Inc,State_Fata,DateChecke,EM_type,EM_date,EM_notes,url,Thumbnail,Confirmed,Deaths,Age_85,Age_80_84,Age_75_79,Age_70_74,Age_65_69,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,Recovered,Active,State_Conf,State_Deat,State_Reco,State_Test,AgedPop,NewCases,NewDeaths,TotalPop,NonHispWhP,BlackPop,AmIndop,AsianPop,PacIslPop,OtherPop,TwoMorPop,HispPop,Wh_Alone,Bk_Alone,AI_Alone,As_Alone,NH_Alone,SO_Alone,Two_More,Not_Hisp,Age_Less15,Age_15_24,Age_25_34,Age_Over75,Agetotal,NonHisp,Age_35_64,Age_65_74,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,geometry
0,1,Autauga,Alabama,AL,01,01001,1.062699,8462.08,89.926440,13.8,3.6,118.9591,1.305141,01/08/2021 11:30:53,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,4705,50,815,1026,1498,2440,2271,85,55,6,2,55601,7587,942,59338,0,0,389230,5080,0,2009678,8050,60,0,55200,41412,10475,159,568,5,41,1012,1528,42437,10565,159,568,32,409,1030,53672,10842,7192,7064,3339,55200,1528,22052,4711,60.0,99.0,210.0,31.0,37.0,29.0,49.0,26.0,59.0,40.0,36.0,30.0,9.0,48.0,107.911728,96.320346,100.000000,2.209382e+09,246839.865479,"POLYGON ((-86.41312 32.70739, -86.41219 32.526..."
1,2,Baldwin,Alabama,AL,01,01003,1.151903,6808.95,78.432452,9.8,3.6,115.4508,1.305141,01/08/2021 11:30:53,Govt Ordered Community Quarantine,4/3/2020 10:41:19 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,14845,171,3949,4792,7373,11410,13141,386,362,51,8,218022,21069,3393,57588,0,0,389230,5080,0,2009678,40665,189,0,208107,172768,19529,1398,1668,9,410,2972,9353,179526,19764,1522,1680,9,2034,3572,198754,37621,23497,23326,16114,208107,9353,82998,24551,189.0,216.0,253.0,123.0,109.0,132.0,222.0,209.0,220.0,210.0,137.0,117.0,42.0,145.0,86.688499,74.029401,120.089286,5.770469e+09,728445.072448,"MULTIPOLYGON (((-87.78878 31.29877, -87.78849 ..."
2,3,Barbour,Alabama,AL,01,01005,2.168525,6486.88,140.669587,30.9,5.2,68.9280,1.305141,01/08/2021 11:30:53,Govt Ordered Community Quarantine,4/3/2020 10:43:21 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,1614,35,422,551,841,1305,1515,74,30,5,2,24881,6788,433,34382,0,0,389230,5080,0,2009678,4634,17,0,25782,11898,12199,63,85,1,86,344,1106,12216,12266,72,96,1,778,353,24676,4517,3092,3675,1814,25782,1106,9864,2820,17.0,22.0,42.0,3.0,2.0,11.0,3.0,22.0,30.0,45.0,11.0,8.0,2.0,6.0,68.325228,56.711409,88.571429,3.258643e+09,307285.154510,"POLYGON ((-85.25609 32.13767, -85.25569 32.135..."
3,4,Bibb,Alabama,AL,01,01007,2.423019,8843.75,214.285714,21.8,4.0,92.3478,1.305141,01/08/2021 11:30:53,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,1981,48,427,488,624,842,1280,35,25,4,1,22400,4400,344,46064,0,0,389230,5080,0,2009678,3661,37,0,22527,16801,4974,8,37,0,0,160,547,17268,5018,8,37,0,9,187,21980,3742,3005,3075,1539,22527,547,9044,2122,37.0,21.0,38.0,3.0,19.0,9.0,20.0,17.0,25.0,30.0,16.0,7.0,14.0,14.0,165.178571,19.047619,NaN,2.310715e+09,227886.963840,"POLYGON ((-87.02685 33.24646, -87.02572 33.209..."
4,5,Blount,Alabama,AL,01,01009,1.452491,8570.19,124.481328,13.2,3.5,101.0645,1.305141,01/08/2021 11:30:53,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,4957,72,866,1459,1776,2802,3330,25,25,6,2,57840,7527,878,50412,0,0,389230,508

We are highlighting some of the features that we wanted to keep for our analysis and also for engineering:

- FatalityRa = County Fatality Rate
- Confirmedb = County Confirmed divided by County Population * 100,000
- DeathsbyPo = County Deaths divided by County Population * 100,000
- Unemployme = Unemployment Rate
- EM_type, EM_date, EM_notes - could be turned into a column about Emergency Declaration
- Confirmed = County Level Confirmed Cases
- Deaths = County Level Deaths
- Beds_Licen = Number of Licensed Beds 
- Beds_Staff = Number of Staffed Beds 
- Beds_ICU = Number of ICU Beds 
- Ventilator = Average Ventilation Used Per Hospital
- POP_ESTIMA = Total Population
- POVALL_201 = Poverty Rate
- Median_Hou - Median Household Income
- Recovered = Number of Recovered Cases
- Active = Number of Active Cases
- AgedPop = Total Population Aged 65 Plus
- NewCases = New cases since yesterday
- NewDeaths = Newdeaths since yesterday
- Wh_Alone, Bk_Alone, AI_Alone, As_Alone, NH_Alone, SO_Alone, Two_More, Not_Hisp = Population numbers of each group
- Age_? = Population numbers of age groups
- NewCasebyP = I think this is new case by percentage
- ICU_Occ, Inpat_Occ = Percent Occupied of ICUs and Inpatient Beds
- Shape_Area, Shape_Length = for mapping



In [54]:
# remove all the rows with Unassigned and Out of [State] designations for county name, as well as Puerto Rico
geo = geo[~geo.Countyname.str.contains("Out of")]
geo = geo[~geo.Countyname.str.contains("Unassigned")]
geo = geo[~geo.ST_Name.str.contains("Puerto Rico")]
geo.drop(geo.tail(7).index, inplace=True)

In [55]:
# remove Day_1 to Day_14 columns, unclear what the days represent
geo.drop(columns=['Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Day_7', 'Day_8', 'Day_9', 'Day_10', 'Day_11', 'Day_12', 'Day_13', 'Day_14'], inplace=True)
# remove state level data columns, as well as ID columns and miscellaneous data
geo.drop(columns=['OBJECTID', 'ST_ID', 'Med_HH_Inc', 'State_Fata', 'DateChecke', 'url', 'Thumbnail', 'State_Conf', 'State_Deat', 'State_Reco', 'State_Test'], inplace=True)
# remove confirmed and deaths by county population and removed notes column for Emergency Management
geo.drop(columns=['Confirmed', 'Deaths', 'EM_notes'], inplace=True)
# remove columns because most values were null
geo.drop(columns=['Recovered', 'Active'], inplace=True)
# Replaced long version of values into shorter abbreviated versions and extracted the date out of the string
geo.EM_type = geo.EM_type.replace({'Govt Ordered Community Quarantine': 'CQ', 'Govt Directed Social Distancing': 'SD'})
geo.EM_date = geo.EM_date.str.extract(r'((\d+)\/(\d{2})\/(\d{4}))')
# Create percentages for races and removed original race columns as well as the other set of racial columns
geo['Wht_Per'] = geo.NonHispWhP / geo.TotalPop
geo['His_Per'] = geo.HispPop / geo.TotalPop
geo['Blk_Per'] = geo.BlackPop / geo.TotalPop
geo['Asn_Per'] = (geo.AsianPop + geo.PacIslPop) / geo.TotalPop
geo['Nat_Per'] = geo.AmIndop / geo.TotalPop
# Grouped ages into different larger categories
geo['Yth_Per'] = (geo.Age_Less15 + geo.Age_15_24) / geo.Agetotal
geo['Adt_Per'] = (geo.Age_25_34 + geo.Age_35_64) / geo.Agetotal
geo['Sen_Per'] = (geo.Age_65_74 + geo.Age_Over75) / geo.Agetotal
# Created some other features
geo['Aged_Per'] = geo.AgedPop / geo.POP_ESTIMA
geo['Staf_Per'] = geo['Beds_Staff'] / geo['Beds_Licen']
geo['Pov_Per'] = geo.POVALL_201 / geo.POP_ESTIMA
geo['Beds_Per'] = geo.Beds_ICU / geo.POP_ESTIMA
geo['Pop_Dens'] = geo['TotalPop'] / geo['Shape__Area']
# Drop columns 
geo.drop(columns=['Age_85', 'Age_80_84', 'Age_75_79', 'Age_70_74', 'Age_65_69', 'Agetotal', 'Age_Less15', 'Age_15_24', 'Age_25_34', 'Age_35_64', 'Age_65_74', 'Age_Over75'], inplace=True)
geo.drop(columns=['Wh_Alone', 'Bk_Alone', 'AI_Alone', 'As_Alone', 'NH_Alone', 'SO_Alone', 'Two_More', 'Not_Hisp', 'NonHispWhP', 'BlackPop', 'AmIndop', 'PacIslPop', 'OtherPop', 'TwoMorPop', 'HispPop', 'NonHisp', 'AsianPop'], inplace=True)
geo.drop(columns=['NewCases', 'NewDeaths', 'AgedPop', 'POVALL_201', 'Unemployed'], inplace=True)
geo.drop(columns=['Beds_Licen', 'Beds_Staff', 'Beds_ICU', 'Ventilator'], inplace=True)

<img src='images/county_conf_by_pop.png'>

<img src='images/county_confirmed.png'>

Normalizing the data ensures that mere comparisons of raw numbers will skew the numbers towards higher population areas. In our feature engineering and selection process, to ensure that communities with lower populations are not left out or misrepresented, we dropped or engineered any such columns.

For confirmed cases and deaths, the dataset already had the following columns, so we naturally dropped the latter two columns:
- Confirmedb = County Confirmed divided by County Population * 100,000
- DeathsbyPo = County Deaths divided by County Population * 100,000
- Confirmed = County Level Confirmed Cases
- Deaths = County Level Deaths

Turning the numbers into percentages allows us to look at the racial distributions within each county:
- `geo[Wht_Per] = geo.NonHispWhP / geo.TotalPop`
- `geo[His_Per] = geo.HispPop / geo.TotalPop`
- `geo[Blk_Per] = geo.BlackPop / geo.TotalPop`
- `geo[Asn_Per] = (geo.AsianPop + geo.PacIslPop) / geo.TotalPop`
- `geo[Nat_Per] = geo.AmIndop / geo.TotalPop`

In terms of voting behavior, we can lump the age groups into bigger categories, and we can remove the 5-year groupings:
- `geo[Yth_Per] = (geo.Age_Less15 + geo.Age_15_24) / geo.Agetotal`
- `geo[Adt_Per] = (geo.Age_25_34 + geo.Age_35_64) / geo.Agetotal`
- `geo[Sen_Per] = (geo.Age_65_74 + geo.Age_Over75) / geo.Agetotal`
- `geo[Aged_Per] = geo.AgedPop / geo.POP_ESTIMA`

Creating a column to represent population density which would be an indication of urban/rural level would be useful, ultimately discovering a metric called RUCC and incorporating that into the dataset:
- `geo['Pop_Dens'] = geo['TotalPop'] / geo['Shape__Area']`

Lastly, we wanted to capture hospitalization and bed data in some ghopefully meaningful way and decided on the following:
- `geo['Staf_Per'] = geo['Beds_Staff'] / geo['Beds_Licen']`
- `geo['Beds_Per'] = geo.Beds_ICU / geo.POP_ESTIMA`


In [57]:
rural_urban = pd.read_excel('data/ru_code.xls')
fip_rur = rural_urban[['FIPS', 'RUCC_2013']]
fip_rur.FIPS = fip_rur.FIPS.astype(str)
geof = geo.merge(fip_rur, how = 'left', on = 'FIPS')

In [102]:
geof.describe()

,FatalityRa,Confirmedb,DeathsbyPo,PCTPOVALL_,Unemployme,POP_ESTIMA,Median_Hou,TotalPop,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,Wht_Per,His_Per,Blk_Per,Asn_Per,Nat_Per,Yth_Per,Adt_Per,Sen_Per,Aged_Per,Staf_Per,Pov_Per,Beds_Per,Pop_Dens,RUCC_2013
count,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3.142000e+03,3142.000000,3.142000e+03,3142.000000,2423.000000,1552.000000,3.142000e+03,3.142000e+03,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3141.000000,2477.000000,3141.000000,3141.000000,3.142000e+03,2825.000000
mean,1.752863,7025.453342,124.294096,15.157829,4.132018,1.041271e+05,52777.610439,1.027699e+05,76.861698,53.976173,65.560210,6.808623e+09,3.714886e+05,0.764951,0.092623,0.089207,0.014375,0.018288,0.311678,0.504682,0.183640,0.141358,0.874776,0.145583,0.000283,6.210886e-05,5.005310
std,1.108569,2846.660721,90.595127,6.130698,1.503802,3.334863e+05,13909.834889,3.299077e+05,81.358994,34.142614,29.709178,5.556968e+10,7.397408e+05,0.201837,0.137899,0.144647,0.030766,0.075532,0.047807,0.032847,0.045859,0.082917,0.459133,0.056476,0.000350,4.004684e-04,2.706416
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,7.500000e+01,-1418.439716,0.000000,0.000000,8.763938e+06,1.563678e+04,0.007278,0.000000,0.000000,0.000000,0.000000,0.080000,0.231646,0.037986,0.000000,0.000000,0.026183,0.000000,2.429692e-09,1.000000
25%,1.012532,5227.382500,60.766296,10.800000,3.100000,1.092650e+04,43679.500000,1.094800e+04,35.825191,34.330776,50.339860,1.860524e+09,2.058000e+05,0.647585,0.021121,0.006330,0.003158,0.001257,0.284786,0.486504,0.154482,0.110546,0.754026,0.105443,0.000083,3.850014e-06,2.000000
50%,1.544236,6897.985000,106.998770,14.100000,3.900000,2.575850e+04,50566.500000,2.573600e+04,68.932516,54.614412,72.340426,2.680092e+09,2.493543e+05,0.839246,0.041010,0.021594,0.006433,0.002771,0.308982,0.505499,0.180380,0.164017,0.938776,0.136557,0.000205,1.044782e-05,6.000000
75%,2.288105,8603.997500,164.100587,18.300000,4.800000,6.782050e+04,58845.500000,6.720900e+04,103.590743,72.141680,88.000071,3.944540e+09,3.367670e+05,0.926928,0.095586,0.098571,0.013624,0.006750,0.333725,0.523774,0.208046,0.195383,1.000000,0.175465,0.000356,2.739642e-05,7.000000
max,13.043478,28456.810000,842.266462,54.000000,19.900000,1.010552e+07,140382.000000,1.009805e+07,1579.586877,1218.285714,200.000000,2.228678e+12,1.796423e+07,1.000000,0.990688,0.874123,0.533333,0.909612,0.611674,0.760000,0.555963,0.522477,20.880000,0.537284,0.004112,1.586326e-02,9.000000


## 2020 Election Data

In [46]:
# upload election data
df2020 = pd.read_csv('data/2020_Results.csv')
df2020.rename(columns={'county_fips': 'FIPS'}, inplace=True)
df2020.FIPS = df2020.FIPS.astype(str)
df2020.FIPS = df2020.FIPS.apply(lambda x: x.zfill(5))
# upload shapefile for USA counties
shapefile = 'data/USA_Counties.shp'
gdf = gpd.read_file(shapefile)
gdf = gdf[~gdf.STATE_NAME.isin(['Puerto Rico', 'Alaska', 'Hawaii'])]
gdf.FIPS = gdf.FIPS.astype(str)
merged = gdf.merge(df2020, on='FIPS', how='left')

In [36]:
# fig, ax = plt.subplots(1, figsize=(20,20))
# merged.plot(column='per_gop', cmap='RdBu_r', linewidth=0.8, ax=ax, edgecolor='black')
# ax.axis('off')
# ax.set_title('Percentage By County for 2020 Election', fontsize=18)
# plt.savefig('images/election2020.png')

<img src='images/election2020.png'>

# Visualizing Combined Data

In [91]:
# combine all the dataframes together
merged = geof.merge(df2020, on='FIPS', how='left')
merged_2 = merged.merge(nymask, on='FIPS', how='left')
merged_3 = merged_2.merge(mask_means, on='FIPS', how='left')
merged_3.to_file("data/combined.geojson", driver='GeoJSON')
# turn geopandas dataframe into pandas dataframe
df = pd.DataFrame(merged_3)

In [92]:
# view column names to select for features
df.columns

Index(['Countyname', 'ST_Name', 'ST_Abbr', 'FIPS', 'FatalityRa', 'Confirmedb',
       'DeathsbyPo', 'PCTPOVALL_', 'Unemployme', 'EM_type', 'EM_date',
       'POP_ESTIMA', 'Median_Hou', 'TotalPop', 'NewCasebyP', 'Inpat_Occ',
       'ICU_Occ', 'Shape__Area', 'Shape__Length', 'geometry', 'Wht_Per',
       'His_Per', 'Blk_Per', 'Asn_Per', 'Nat_Per', 'Yth_Per', 'Adt_Per',
       'Sen_Per', 'Aged_Per', 'Staf_Per', 'Pov_Per', 'Beds_Per', 'Pop_Dens',
       'RUCC_2013', 'state_name', 'county_name', 'votes_gop', 'votes_dem',
       'total_votes', 'diff', 'per_gop', 'per_dem', 'per_point_diff', 'NEVER',
       'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS', 'ind_mask', 'oth_mask'],
      dtype='object')

In [94]:
# convert continuous into categorical variable
df['per_dem_cat'] = df['per_dem']
choices = [0,1,2,3]
condition = [(df['per_dem_cat']) <= .25, 
             (df['per_dem_cat'] > .25) & (df['per_dem_cat'] <= .50),
             (df['per_dem_cat'] > .50) & (df['per_dem_cat'] <= .75),
             (df['per_dem_cat']) > .75]
df['per_dem_cat'] = np.select(condition, choices, df['per_dem_cat'])

In [95]:
# Create scatterplots of features
df.FIPS = df.FIPS.astype(int)
features = df[['FIPS', 'FatalityRa', 'Confirmedb',
       'DeathsbyPo', 'PCTPOVALL_', 'Unemployme', 'POP_ESTIMA',
       'Median_Hou', 'TotalPop', 'NewCasebyP', 'Inpat_Occ', 'ICU_Occ',
       'Wht_Per', 'His_Per', 'Blk_Per', 'Asn_Per', 'Nat_Per', 'Yth_Per', 'Adt_Per',
       'Sen_Per', 'Staf_Per', 'Aged_Per', 'Pov_Per', 'Beds_Per', 'Pop_Dens',
       'RUCC_2013', 'ind_mask', 'oth_mask', 'NEVER', 'RARELY',
       'SOMETIMES', 'FREQUENTLY', 'ALWAYS']]
target = df['per_dem_cat']
feat_col = features.columns

In [103]:
# con = pd.melt(df, id_vars='per_dem_cat', value_vars=feat_col)
# g = sns.FacetGrid(con, col='variable', col_wrap=3, sharex=False, sharey=False, height=4)
# g = g.map(sns.scatterplot, 'value', 'per_dem_cat', color='darkorange')
# g.set_xticklabels(rotation=45)
# plt.savefig('images/scatterplot.png')

<img src='images/scatterplot.png'>

In [100]:
# con = pd.melt(df, id_vars='per_dem_cat', value_vars=feat_col)
# g = sns.FacetGrid(con, col='variable', col_wrap=3, sharex=False, sharey=False, height=4)
# g = g.map(sns.regplot, 'value', 'per_dem_cat', color='dodgerblue')
# g.set_xticklabels(rotation=45)
# plt.savefig('images/regplot.png')

<img src='images/regplot.png'>

In [101]:
# con_2 = pd.melt(df, value_vars = feat_col)
# g = sns.FacetGrid(con_2, col='variable', col_wrap=3, sharex=False, sharey=False, height=4)
# g = g.map(sns.distplot, 'value', color='mediumslateblue')
# g.set_xticklabels(rotation=45)
# plt.savefig('images/distplot.png')

<img src='images/distplot.png'>